In [13]:
import tetris
import numpy as np
import time
import copy
figures = [
        [[1, 5, 9, 13], [4, 5, 6, 7]],
        [[4, 5, 9, 10], [2, 6, 5, 9]],
        [[6, 7, 9, 10], [1, 5, 6, 10]],
        [[1, 2, 5, 9], [0, 4, 5, 6], [1, 5, 9, 8], [4, 5, 6, 10]],
        [[1, 2, 6, 10], [5, 6, 7, 9], [2, 6, 10, 11], [3, 5, 6, 7]],
        [[1, 4, 5, 6], [1, 4, 5, 9], [4, 5, 6, 9], [1, 5, 6, 9]],
        [[1, 2, 5, 6]],
    ]


def column_height(field): # from top to bottom
    h = []
    for j in range(10):
        column = [field[i][j] for i in range(20)]
        height = 0
        while height < 20 and column[height] == 0:
            height += 1
        h.append(20 - height)
    return h


def maximum_height(field):
    return(max(column_height(field)))

def column_difference(field):# absolute difference between adjacent columns
    df=[]
    h=column_height(field)
    for j in range(9):
        df.append(abs(h[j+1]-h[j]))
    return(df)

def bumpiness(field):
    return(sum(column_difference(field)))

def holes(field):
    L = 0
    h  =column_height(field)
    for j in range(10):
        for i in range(20-h[j],20):
            if field[i][j] == 0:
                L+=1
    return(L)

def evaluate(W, field): 
    #W=[w1, ..., w21] vector of parameters to tune 
    h=column_height(field)
    dh=column_difference(field)
    L=holes(field)
    H=maximum_height(field)
    S = 0
    for k in range (len(h)):
        S += h[k] * W[k]
    for k in range (len(dh)):
        S += dh[k] * W[10+k]
    S += W[19] * L
    S += W[20] * H
    return(S)


def evaluate_best_move(W,field,type):
    L=[]
    score=[]
    n_rot = len(figures[type])
    
    for k in range (n_rot):
        for col in range (-5,10):
            game_copy = tetris.Tetris(20,10)
            
            game_copy.field = [[cell for cell in row] for row in field]

            game_copy.new_figure(type)
            game_copy.rotate(k)
            game_copy.go_side(col) 
            

            if not game_copy.intersects():
                game_copy.go_space()
                score.append(evaluate(W,game_copy.field))
                L.append((col,k))
    if len(L)>0:
        best_move = score.index(min(score))
        return(L[best_move])
    else : 
        return((0,0))

def get_next_state(tetris_instance, figure_type):
    '''Returns all possible moves as { (x, rotation): features }'''
    states = {}
    num_rotations = len(figures[figure_type])  # Number of rotations for the given piece type

    for rotation in range(num_rotations):  # Iterate through all possible rotations
        # Get the current rotation of the piece
        piece = figures[figure_type][rotation]

        # Calculate the x-boundaries of the piece
        min_x = min(p % 4 for p in piece)
        max_x = max(p % 4 for p in piece)

        for x in range(-min_x, tetris_instance.width - max_x):  # Iterate through all possible x positions
            tetris_copy = copy.deepcopy(tetris_instance)  # Copy the Tetris game state to simulate moves
            tetris_copy.new_figure(figure_type)  # Initialize a new piece
            tetris_copy.figure.x = x  # Set x-position
            tetris_copy.figure.rotation = rotation  # Set rotation

            # Simulate dropping the piece
            tetris_copy.go_space()

            # Extract features after the piece placement
            temp_field = tetris_copy.field
            lines_cleared = tetris_copy.score - tetris_instance.score
            holes_count = holes(temp_field)
            bumpiness_value = bumpiness(temp_field)
            total_height = sum(column_height(temp_field))

            # Store the results for this move
            states[(x, rotation)] = [lines_cleared, holes_count, bumpiness_value, total_height]

    return states
def get_state_size():
    '''Renvoie la taille du vecteur d’état'''
    return 4


def play_move(env, x, rotation):
    curr_lines=env.score
    reward, game_over = env.play(x, rotation)
    lines_cleared = env.score - curr_lines
    column_heights = column_height(env.field)
    bumpiness_value = bumpiness(env.field)
    total_height = sum(column_heights)
    holes_count = holes(env.field)
    new_state = (lines_cleared, holes_count, bumpiness_value, total_height)
    return reward, new_state, game_over 

def simulation(W):
    
    game = tetris.Tetris(20, 10)
    while game.state != "gameover":

        type = np.random.randint(0,6)
        game.new_figure(type)

        col, rot = evaluate_best_move(W,game.field,type)
        game.rotate(rot)
        game.go_side(col)
        if game.intersects():
            game.state="gameover"
        
        else:
            game.go_space()

    return(game.score)

In [ ]:
import random
import cv2
import numpy as np
from PIL import Image
from time import sleep
import copy

# Pour le modèle Keras
from keras.models import Sequential, load_model
from keras.layers import Dense,Input,Conv2D,Flatten
from collections import deque

from IPython.display import display, clear_output
from PIL import Image
import numpy as np
import cv2
class DQNAgent:
    def __init__(self, state_size, board_shape=None, model_type='dense',
                 mem_size=10000, discount=0.95, epsilon=1, epsilon_min=0,
                 epsilon_stop_episode=0, n_neurons=[32, 32],
                 activations=['relu', 'relu', 'linear'], loss='mse',
                 optimizer='adam', replay_start_size=None, modelFile=None):

        if model_type not in ['dense', 'conv']:
            raise ValueError("model_type must be 'dense' or 'conv'")

        self.model_type = model_type
        self.state_size = state_size
        self.board_shape = board_shape  # Ex: (20, 10) pour Tetris

        self.mem_size = mem_size
        self.memory = deque(maxlen=mem_size)
        self.discount = discount
        if epsilon_stop_episode > 0:
            self.epsilon = epsilon
            self.epsilon_min = epsilon_min
            self.epsilon_decay = (self.epsilon - self.epsilon_min) / epsilon_stop_episode
        else:
            self.epsilon = 0

        self.n_neurons = n_neurons
        self.activations = activations
        self.loss = loss
        self.optimizer = optimizer
        if not replay_start_size:
            replay_start_size = mem_size / 2
        self.replay_start_size = replay_start_size

        if modelFile is not None:
            self.model = load_model(modelFile)
        else:
            self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(self.n_neurons[0], input_dim=self.state_size, activation=self.activations[0]))
        for i in range(1, len(self.n_neurons)):
            model.add(Dense(self.n_neurons[i], activation=self.activations[i]))
        model.add(Dense(1, activation=self.activations[-1]))
        model.compile(loss=self.loss, optimizer=self.optimizer)
        return model

    def predict_value(self, state):
        state = np.reshape(state, [1, self.state_size])  # Reshape to (1, 4)
        return self.model.predict(state, verbose=0)[0]


    def act(self, state):
        '''Retourne le score attendu pour un état (avec exploration)'''
        state = np.reshape(state, [1, self.state_size])
        if random.random() <= self.epsilon:
            return self.random_value()
        else:
            return self.predict_value(state)

    def best_state(self, states):
        '''Retourne le meilleur état (celui qui a le score attendu le plus élevé)
        à partir d'une collection d'états.
        '''
        max_value = None
        best_state = None
        if random.random() <= self.epsilon:
            return random.choice(list(states))
        else:
            for state in states:
                value = self.predict_value(np.reshape(state, [1, self.state_size]))
                if max_value is None or value > max_value:
                    max_value = value
                    best_state = state
        return best_state

    def train(self, batch_size=32, epochs=3):
        '''Entraîne le modèle sur un batch de transitions'''
        n = len(self.memory)
        if n >= self.replay_start_size and n >= batch_size:
            batch = random.sample(self.memory, batch_size)
            # Prédire les scores pour les prochains états en batch
            next_states = np.array([x[1] for x in batch])
            next_qs = [x[0] for x in self.model.predict(next_states)]
            x = []
            y = []
            for i, (state, _, reward, done) in enumerate(batch):
                if not done:
                    new_q = reward + self.discount * next_qs[i]
                else:
                    new_q = reward
                x.append(state)
                y.append(new_q)
            self.model.fit(np.array(x), np.array(y), batch_size=batch_size, epochs=epochs, verbose=0)
            if self.epsilon > self.epsilon_min:
                self.epsilon -= self.epsilon_decay
    def update_epsilon(self):
      if self.epsilon > self.epsilon_min:
          self.epsilon -= self.epsilon_decay
          self.epsilon = max(self.epsilon, self.epsilon_min)

    def add_to_memory(self, current_state, action, reward, next_state, done):
        '''Ajoute une transition (state, action, reward, next_state, done) à la mémoire'''
        self.memory.append((current_state, action, reward, next_state, done))

    def save_model(self, name):
        '''Sauvegarde le modèle entraîné'''
        self.model.save(name)

In [15]:
import os
from tqdm import tqdm  # Pour afficher une barre de progression (optionnel)
env = tetris.Tetris(20,10)

# Paramètres d'entraînement
episodes = 1600                # nombre total d'épisodes
max_steps = None         # nombre maximum d'actions par épisode (None = illimité)
epsilon_stop_episode = 1400    # épisode auquel l'exploration aléatoire s'arrête
mem_size = 1000                # taille max de la mémoire (nombre d'étapes stockées)
discount = 0.95                # facteur de discount pour Q-learning
batch_size = 128               # taille du batch pour l'entraînement
epochs = 1                     # nombre d'epochs par entraînement
render_every = 50              # affichage du jeu tous les x épisodes
render_delay = None            # délai entre les frames (None = sans délai)
replay_start_size = 1000       # nombre minimal de transitions stockées pour démarrer l'entraînement
train_every = 1                # entraînement de l'agent tous les x épisodes
n_neurons = [32, 32, 32]       # nombre de neurones par couche cachée
activations = ['relu', 'relu', 'relu', 'linear']
save_best_model = True         # sauvegarder le meilleur modèle dans "best.keras"
model_file = "sample.keras"

In [16]:
model_file = "best.keras"
if os.path.exists(model_file):
    print(f"Chargement du modèle existant : {model_file}")
    agent = DQNAgent(get_state_size(), board_shape=(20, 10),
                     n_neurons=n_neurons, activations=activations,
                     epsilon_stop_episode=epsilon_stop_episode, mem_size=mem_size,
                     discount=discount, replay_start_size=replay_start_size,
                     modelFile=model_file)
else:
    agent = DQNAgent(get_state_size(), board_shape=(20, 10),
                     n_neurons=n_neurons, activations=activations,
                     epsilon_stop_episode=epsilon_stop_episode, mem_size=mem_size,
                     discount=discount, replay_start_size=replay_start_size,
                     model_type='dense')

scores = []
best_score = 0

for episode in tqdm(range(episodes)):
    current_state = env.reset()  # État initial (vecteur de features)
    #print(env.field)
    done = False
    steps = 0
    render = render_every and episode % render_every == 0

    while not done and (max_steps is None or steps < max_steps):
        figure_type = np.random.randint(0, 7)  # Random figure type
        next_states = get_next_state(env, figure_type)  # Call the function

        if not next_states:
            break

        # Exploration or exploitation
        if random.random() <= agent.epsilon:
            # Random exploration
            random_key = random.choice(list(next_states.keys()))
            best_action, next_state_features = random_key, next_states[random_key]
        else:
            # Exploitation using DQN
            best_state = agent.best_state(list(next_states.values()))

            # Find the action tuple corresponding to the best feature tuple
            best_action = None
            for action, features in next_states.items():
                if features == best_state:
                    best_action = action
                    next_state_features = features
                    break

            if best_action is None:
                raise ValueError("No action found for the best state.")
        # Perform the action and retrieve reward and game status
        x, rotation = best_action
        reward, new_state, done = play_move(env, x, rotation)

        # Add the transition to memory
        agent.add_to_memory(current_state, best_action, reward, new_state, done)

        current_state = new_state
        steps += 1

        # Entraînement en ligne (optionnel : pour apprendre directement après chaque step)
        if episode >= agent.replay_start_size:
            agent.train(batch_size=batch_size, epochs=epochs)

    scores.append(env.reward)

    # Mise à jour d'epsilon à la fin de chaque épisode
    agent.update_epsilon()

    # Affichage résumé tous les X épisodes
    if (episode + 1) % 50 == 0:
        avg_score = sum(scores[-50:]) / len(scores[-50:])
        print(f"Episode {episode+1}/{episodes} - Moyenne score (50 derniers) : {avg_score:.2f} - Epsilon : {agent.epsilon:.4f}")

    # Sauvegarder le meilleur modèle
    if save_best_model and env.reward > best_score:
        print(f"✅ Nouveau meilleur modèle (score = {env.reward}, episode = {episode})")
        best_score = env.reward
        agent.save_model("best.keras")

Chargement du modèle existant : best.keras


  0%|          | 0/1600 [00:00<?, ?it/s]

✅ Nouveau meilleur modèle (score = 18, episode = 0)


  0%|          | 2/1600 [00:00<01:57, 13.63it/s]

✅ Nouveau meilleur modèle (score = 21, episode = 2)


...

Episode 950/1600 - Moyenne score (50 derniers) : 17.70 - Epsilon : 0.3214


 62%|██████▎   | 1000/1600 [57:18<1:52:07, 11.21s/it]

Episode 1000/1600 - Moyenne score (50 derniers) : 17.32 - Epsilon : 0.2857


 62%|██████▎   | 1000/1600 [57:20<34:24,  3.44s/it]  


InvalidArgumentError: Graph execution error:

Detected at node sequential_1/dense_1/Relu defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\anyone\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "C:\Users\anyone\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\Users\anyone\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "C:\Users\anyone\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tornado\platform\asy